In [ ]:
#remove skin from product images
# file: save_ckp_frozen.h5 remove skin model

In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import tensorflow as tf
import sys
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

#config = ConfigProto(device_count={'GPU':0,'CPU':1})
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)


In [2]:
f = sys.argv[1]
saved = load_model("/home/hadil/final/save_ckp_frozen.h5")

2022-05-18 00:04:32.771167: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
class fashion_tools(object):
    def __init__(self,imageid,model,version=1.1):
        self.imageid = imageid
        self.model   = model
        self.version = version
        
    def get_dress(self,stack=False):
        """limited to top wear and full body dresses (wild and studio working)"""
        """takes input rgb----> return PNG"""
        name =  self.imageid
        file = cv2.imread("/home/hadil/final/hmgoepprod.jpeg")
        file = tf.image.resize_with_pad(file,target_height=512,target_width=512)  #Redimensionne et remplit une image à une largeur et une hauteur cibles.
        rgb  = file.numpy()
        file = np.expand_dims(file,axis=0)/ 255
        seq = self.model.predict(file)
        seq = seq[3][0,:,:,0]
        seq = np.expand_dims(seq,axis=-1)
        c1x = rgb*seq
        c2x = rgb*(1-seq)
        
        cfx = c1x+c2x
        dummy = np.ones((rgb.shape[0],rgb.shape[1],1))
        #rgbx = np.concatenate((rgb,dummy*255),axis=-1)
        rgbs = np.concatenate((cfx,seq*255.),axis=-1)
        if stack:
            #stacked = np.hstack((rgbs,rgbs))
            stacked=rgbs
            return stacked
        else:
            return rgbs
        
        
    def get_patch(self):
        return None


In [6]:
api    = fashion_tools(f,saved)
image_ = api.get_dress(stack=True)
cv2.imshow('image_',image_)
cv2.imwrite('xc.jpg',image_)
cv2.waitKey(0)

1/1 [==============================] - 2s 2s/step


-1